In [1]:
!pip install openpyxl

In [2]:
!pip install --upgrade jupyter ipywidgets

In [3]:
import torch
import pandas as pd
from tqdm.notebook import tqdm
import csv

In [4]:
csv_file ='/u1/mdr614/On the compleness of review comments/Notebook_on the compleness/BERT_CONFU_AUTHOR/train_bert_author.xlsx' 
df = pd.read_excel(csv_file)
#df = pd.read_csv(csv_file)
df.head()

,sl,Project,pullNumber,author,comment,path,label
0,1,beam,16917,yeandy,Renamed `PytorchModel` to `PytorchModelSpec` s...,sdks/python/apache_beam/ml/inference/api.py,1
1,2,flink,13648,curcur,But I can think of a possible case that might ...,flink-runtime/src/main/java/org/apache/flink/r...,1
2,3,seatunnel,4683,sunxiaojian,> Please add this to `pom.xml` not in submodul...,seatunnel-engine/seatunnel-engine-storage/imap...,1
3,4,bookkeeper,1225,reddycharan,changing,bookkeeper-server/src/test/java/org/apache/boo...,0
4,5,flink,3621,dawidwys,done,flink-libraries/flink-cep/src/main/java/org/ap...,0


In [5]:
df.label.value_counts()

0    22624
1     8095
Name: label, dtype: int64

In [6]:
possible_labels = df.label.unique()
possible_labels

array([1, 0])

In [7]:
label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index

In [8]:
label_dict = {'1': 1, '0': 0}
label_dict

{'1': 1, '0': 0}

In [9]:
df['label'] = df.label.replace(label_dict)

In [10]:
df.head()

,sl,Project,pullNumber,author,comment,path,label
0,1,beam,16917,yeandy,Renamed `PytorchModel` to `PytorchModelSpec` s...,sdks/python/apache_beam/ml/inference/api.py,1
1,2,flink,13648,curcur,But I can think of a possible case that might ...,flink-runtime/src/main/java/org/apache/flink/r...,1
2,3,seatunnel,4683,sunxiaojian,> Please add this to `pom.xml` not in submodul...,seatunnel-engine/seatunnel-engine-storage/imap...,1
3,4,bookkeeper,1225,reddycharan,changing,bookkeeper-server/src/test/java/org/apache/boo...,0
4,5,flink,3621,dawidwys,done,flink-libraries/flink-cep/src/main/java/org/ap...,0


In [11]:
from sklearn.model_selection import train_test_split

In [12]:
X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  df.label.values, 
                                                  test_size=0.15, 
                                                  random_state=17, 
                                                  stratify=df.label.values)

In [13]:
X_train

array([20974,  9795, 25388, ..., 12691,  7531, 23254])

In [14]:
df['data_type'] = ['not_set']*df.shape[0]

In [15]:
df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

In [16]:
df

,sl,Project,pullNumber,author,comment,path,label,data_type
0,1,beam,16917,yeandy,Renamed `PytorchModel` to `PytorchModelSpec` s...,sdks/python/apache_beam/ml/inference/api.py,1,train
1,2,flink,13648,curcur,But I can think of a possible case that might ...,flink-runtime/src/main/java/org/apache/flink/r...,1,train
2,3,seatunnel,4683,sunxiaojian,> Please add this to `pom.xml` not in submodul...,seatunnel-engine/seatunnel-engine-storage/imap...,1,train
3,4,bookkeeper,1225,reddycharan,changing,bookkeeper-server/src/test/java/org/apache/boo...,0,train
4,5,flink,3621,dawidwys,done,flink-libraries/flink-cep/src/main/java/org/ap...,0,train
...,...,...,...,...,...,...,...,...
30714,30715,apex-core,568,sandeshh,"Not required, changing it back.",engine/src/main/java/com/datatorrent/stram/Str...,0,train
30715,30716,apex-core,568,sandeshh,Done,bufferserver/src/main/java/com/datatorrent/buf...,0,train
30716,30717,apex-core,569,sanjaypujare,fixed,engine/src/test/java/com/datatorrent/stram/cli...,0,train
30717,30718,apex-core,569,sanjaypujare,fixed,engine/src/test/java/com/datatorrent/stram/cli...,0,train


In [17]:
df.groupby(['label', 'label', 'data_type']).count()

sl  Project  pullNumber  author  comment   path
label label data_type                                                    
0     0     train      19230    19230       19230   19230    19230  19230
            val         3394     3394        3394    3394     3394   3394
1     1     train       6881     6881        6881    6881     6881   6881
            val         1214     1214        1214    1214     1214   1214

In [18]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

In [19]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)

In [20]:
df[df.data_type=='train'].comment.values

array(['Renamed `PytorchModel` to `PytorchModelSpec` since we renamed the old `ModelSpec` to `ModelLoader`. What do you think @ryanthompson591 ?',
       'But I can think of a possible case that might have a problem:\n\nThread1: create view\nThread2: release view, create view\nThread1: release view (the release is way too slow)\n\nLet me think of this tonight. In the worst case, we can always release view before create a new one.',
       '> Please add this to `pom.xml` not in submodule pom please.\n\nMove to where?',
       ..., 'fixed', 'fixed', 'Done'], dtype=object)

In [21]:
df.iloc[X_val[0]]

sl                                                        13411
Project                                                   flink
pullNumber                                                 8468
author                                                walterddr
comment       I think the special case can be put outside of...
path          flink-table/flink-table-planner-blink/src/main...
label                                                         1
data_type                                                   val
Name: 13410, dtype: object

In [22]:
df.iloc[y_val[0]]

sl                                                            2
Project                                                   flink
pullNumber                                                13648
author                                                   curcur
comment       But I can think of a possible case that might ...
path          flink-runtime/src/main/java/org/apache/flink/r...
label                                                         1
data_type                                                 train
Name: 1, dtype: object

In [23]:
X_val[0]

13410

In [268]:
# from transformers import AutoTokenizer
# import torch
# import pandas as pd

# # Replace 'model_name' with the name of the pre-trained model you're using
# model_name = 'bert-base-uncased'
# tokenizer = AutoTokenizer.from_pretrained(model_name)

# # Assuming you have a DataFrame named 'df'
# for index, row in df.iterrows():
#     data_type = row['data_type']
#     message = row['comment']
#     label = row['label']

#     encoded_data = tokenizer.encode_plus(
#         text=message,
#         add_special_tokens=True,
#         return_attention_mask=True,
#         pad_to_max_length=True,
#         max_length=256,
#         return_tensors='pt'
#     )

#     input_ids = encoded_data['input_ids']
#     attention_mask = encoded_data['attention_mask']
    
#     print(f"Processing row {index + 1} - Data Type: {data_type}, Message: {message}, Label: {label}")
    
#     # Here you can proceed with storing or processing 'input_ids', 'attention_mask', and 'label'
 

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/home/mdr614/anaconda3/envs/toxic_gru/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2619: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Processing row 1 - Data Type: train, Message: done, Label: 0
Processing row 2 - Data Type: val, Message: done, Label: 0
Processing row 3 - Data Type: train, Message: done, Label: 0
Processing row 4 - Data Type: train, Message: done, Label: 0
Processing row 5 - Data Type: train, Message: done
, Label: 0
Processing row 6 - Data Type: train, Message: It isn't usually necessary to pass this in when run_benchmarks is called, as the default suffices for all existing CI use cases.  I've now improved it to detect the current repo using `git remote get-url origin` rather than hardcoding `apache/geode`.  Does this resolve your concern?, Label: 1
Processing row 7 - Data Type: train, Message: Done, Label: 0
Processing row 8 - Data Type: val, Message: I am not sure if client delta put will generate region version less than current local cache version. However, use ">" might be more clear in this case. , Label: 1
Processing row 9 - Data Type: train, Message: uh, nope... I was just guarding against t

Processing row 1132 - Data Type: train, Message: yes, fixed, Label: 0
Processing row 1133 - Data Type: train, Message: I'm not seeing any Alert related information in this class, only URI-s and http(s) related configuration. Even MetricsUri sounds too specific if we consider what's inside., Label: 1
Processing row 1134 - Data Type: val, Message: It has the same structure as for blueprints. For blueprints it is modelled by the BlueprintMpackInstanceEntity, BlueprintServiceEntity, BlueprintMpackConfigEntities. 

Currently we are interested in the stack id's only, later also in configurations and service descriptors associated with mpacks (I saw configs are already dumped as json too).

Do you think it is worth normalizing? (would mean 3 more tables, later maybe more) This information wouldn't be used outside of the scope of the replayed request., Label: 1
Processing row 1135 - Data Type: train, Message: Fixed., Label: 0
Processing row 1136 - Data Type: train, Message: Fixed., Label: 0
Pr

Processing row 2227 - Data Type: val, Message: That's right.  It's not legal to not provide the full primary key and use a custom if clause.

So there's possibly the case where enough parameters are provided to fulfill the number of primary key parameters, but some of those parameters were meant for the custom if clause.  In this case it's likely it won't match the primary key definition and you'll get an error which explains that he parameters don't match the primary key, which is technically correct.  If they do somehow match, there's not much we can do at compile time (I think?).  In any case, you'll get an error at run time, which is similar to what would happen if you don't provide enough parameters to match the query bind markers., Label: 1
Processing row 2228 - Data Type: train, Message: should we add `warn` for that case or fail-silently?, Label: 1
Processing row 2229 - Data Type: train, Message: There were some failures of https://jenkins-drivers.build.dsinternal.org/job/datas

Processing row 3412 - Data Type: train, Message: > actually works as intended.

I have no issue  with the testing you suggested. However, I don't know whether we will be able to tell if the file was *actually* purged from the OS page cache as the FADVISE directive is advice., Label: 1
Processing row 3413 - Data Type: train, Message: That's fair, a better place for the defined Property is in [CompactionConfigurer](https://github.com/apache/accumulo/blob/main/core/src/main/java/org/apache/accumulo/core/client/admin/compaction/CompactionConfigurer.java). That's user-pluggable via a table [property](https://github.com/apache/accumulo/blob/main/core/src/main/java/org/apache/accumulo/core/conf/Property.java#L1007). I'm not sure why the CompactionConfigurer class isn't in the SPI package., Label: 1
Processing row 3414 - Data Type: val, Message: I wonder if it is better to try and create some fake properties for the tests or something to check the resolve method vs leaving in stubs for non exi

Processing row 4435 - Data Type: train, Message: finished, Label: 0
Processing row 4436 - Data Type: val, Message: finished, Label: 0
Processing row 4437 - Data Type: val, Message: Thanks. 1.7.10 is several years old and I don't know why maven suggested that I pick that up. , Label: 0
Processing row 4438 - Data Type: val, Message: I'm not really convinced that mis-rep count really means much and it is worth ordering them further, as the container already has enough replicas. We want these to start with a lower priority than anything else (under rep, decommission), which setting it to 6 does. After that, we can only lower the priority further. If we have a mis rep count of 1 or 2 for a 3-2 and a 6-3 container, which is higher priority? I'm not sure about a good way to do this., Label: 1
Processing row 4439 - Data Type: train, Message: Could you tell me why it would be more useful? The process is short-term and hence the LeadershipManager will be temporary. The cost of creating a such cl

In [24]:
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].comment.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].comment.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/home/mdr614/anaconda3/envs/toxic_gru/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2619: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [25]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [26]:
len(dataset_train)

26111

In [27]:
len(dataset_val)

4608

In [28]:
from transformers import BertForSequenceClassification
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [29]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [30]:
batch_size = 16

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

In [31]:
from transformers import get_linear_schedule_with_warmup

In [32]:
from torch.optim import AdamW
optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)

In [33]:
epochs = 5

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [34]:
import numpy as np
from sklearn.metrics import f1_score

In [35]:
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

In [36]:
def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [37]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [38]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cuda


In [39]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [40]:
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)             
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/1632 [00:00<?, ?it/s]


Epoch 1
Training loss: 0.2395279289373909
Validation loss: 0.19959874889213503
F1 Score (Weighted): 0.9112990453593286


Epoch 2:   0%|          | 0/1632 [00:00<?, ?it/s]


Epoch 2
Training loss: 0.17180071954834264
Validation loss: 0.26256128419683794
F1 Score (Weighted): 0.9093452163843835


...


Epoch 3
Training loss: 0.1288558309276114
Validation loss: 0.3152433574806916
F1 Score (Weighted): 0.9073013828192682


Epoch 4:   0%|          | 0/1632 [00:00<?, ?it/s]


Epoch 4
Training loss: 0.09207398843366665
Validation loss: 0.38698983247053953
F1 Score (Weighted): 0.9072636489031325


Epoch 5:   0%|          | 0/1632 [00:00<?, ?it/s]


Epoch 5
Training loss: 0.06854368936006615
Validation loss: 0.4559595732719117
F1 Score (Weighted): 0.9058740760258193


In [41]:
val_loss, predictions, true_vals = evaluate(dataloader_validation)

In [42]:
preds_flat = np.argmax(predictions, axis=1).flatten()

In [43]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [68]:
model.load_state_dict(torch.load('finetuned_BERT_MYDATA.model', map_location=torch.device('cpu')))

<All keys matched successfully>

In [69]:
_, predictions, true_vals = evaluate(dataloader_validation)

In [70]:
accuracy_per_class(predictions, true_vals)

Class: 0
Accuracy: 2850/3394

Class: 1
Accuracy: 1044/1214



In [71]:
len(predictions), len(true_vals)

(4608, 4608)

In [72]:
from sklearn import metrics

In [73]:
print("Accuracy:",metrics.accuracy_score(preds_flat, true_vals))
print("Precision:",metrics.precision_score(preds_flat, true_vals))
print("Recall:",metrics.recall_score(preds_flat, true_vals))
print("f1 score", metrics.f1_score(preds_flat, true_vals, average='weighted'))

Accuracy: 0.9051649305555556
Precision: 0.8443163097199341
Recall: 0.805184603299293
f1 score 0.9044557850852919


### Testing

In [92]:
xlsx_file ='/u1/mdr614/On the compleness of review comments/Notebook_on the compleness/BERT_CONFU_AUTHOR/test_bert_author.xlsx' 
df_test = pd.read_excel(xlsx_file)
#df_test = pd.read_csv(xlsx_file)
df_test.head()

,Project,pullNumber,author,comment,path,actual,prediction
0,pandas-dev_pandas,21318,pyryjook,"Sure, Iâ€™ll make the change. Have to say that...",pandas/tests/io/json/test_json_table_schema.py,0,0
1,elastic_elasticsearch,42982,original-brownbear,Right ... obviously in my local testing I just...,plugins/repository-azure/qa/microsoft-azure-st...,0,0
2,pandas-dev_pandas,43056,debnathshoham,"Hi @phofl , @attack68 .. I will work on these ...",pandas/tests/reshape/merge/test_merge.py,1,0
3,grpc_grpc,21988,lidizheng,Good idea for the optimization. Updated to fet...,src/python/grpcio/grpc/experimental/aio/_chann...,0,0
4,apache_incubator-mxnet,14359,haojin2,done,src/operator/contrib/index_copy.cc,0,0


In [93]:
possible_labels = df_test.actual.unique()
possible_labels

array([0, 1])

In [94]:
label_dict = {0: 0, 1: 1}
label_dict

{0: 0, 1: 1}

In [95]:
label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index

In [96]:
df_test['label'] = df_test.actual.replace(label_dict)

In [97]:
# from transformers import AutoTokenizer
# import torch
# import pandas as pd

# # Replace 'model_name' with the name of the pre-trained model you're using
# model_name = 'bert-base-uncased'
# tokenizer = AutoTokenizer.from_pretrained(model_name)

# df_test = df_test[10000:]

# # Assuming you have a DataFrame named 'df'
# for index, row in df_test.iterrows():
#     #data_type = row['data_type']
#     message = row['comment']
#     label = row['actual']

#     encoded_data = tokenizer.encode_plus(
#         text=message,
#         add_special_tokens=True,
#         return_attention_mask=True,
#         pad_to_max_length=True,
#         max_length=256,
#         return_tensors='pt'
#     )

#     input_ids = encoded_data['input_ids']
#     attention_mask = encoded_data['attention_mask']
    
#     print(f"Processing row {index + 1}, Message: {message}, Label: {label}")
    
#     # Here you can proceed with storing or processing 'input_ids', 'attention_mask', and 'label'
 

In [98]:
# encoded_test_val = tokenizer.batch_encode_plus(
#     df_test.comment.values, 
#     add_special_tokens=True, 
#     return_attention_mask=True, 
#     pad_to_max_length=True, 
#     max_length=256, 
#     return_tensors='pt'
# )

encoded_test_val = tokenizer.batch_encode_plus(
    df_test.comment.values.tolist(),  # Convert NumPy array to list
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)


/home/mdr614/anaconda3/envs/toxic_gru/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2619: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [99]:
input_ids_test = encoded_test_val['input_ids']
attention_masks_test = encoded_test_val['attention_mask']
labels_test = torch.tensor(df_test.label.values)

In [100]:
dataset_test = TensorDataset(input_ids_test, attention_masks_test,labels_test)
batch_size = 8
dataloader_test = DataLoader(dataset_test, 
                                   sampler=SequentialSampler(dataset_test), 
                                   batch_size=batch_size)

In [101]:
def test_model(dataloader_test):

    model.eval()
    
    predictions = []
    
    for batch in dataloader_test:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                'attention_mask': batch[1],
                'labels':         batch[2],
                }  
        with torch.no_grad():        
            outputs = model(**inputs)
            
        logits = outputs[1]

        logits = logits.detach().cpu().numpy()
        predictions.append(logits)
    
    predictions = np.concatenate(predictions, axis=0)
    return predictions

In [102]:
pred_test = test_model(dataloader_test) 
preds_flat_test = np.argmax(pred_test, axis=1).flatten()
print(preds_flat_test)

[0 0 1 ... 1 1 1]


In [103]:
import pandas as pd
pred_data = pd.DataFrame(preds_flat_test, columns = ["prediction"])
pd_cont = pd.concat([df_test["comment"], pred_data], axis = 1)
pd_cont.to_csv("test_bert_author_prediction.csv")

In [163]:
len(pred_data)

86861

In [105]:
len(X_val)

16644

In [106]:
pred_data

,prediction
0,2
1,0
2,2
3,2
4,1
...,...
1010,1
1011,1
1012,2
1013,0


In [ ]:
df_slice = df_test["message"]
df_slice.reset_index()

In [ ]:
pd_cont_rst = pd.concat([df_slice.reset_index(), pred_data], axis = 1)
pd_cont_rst.to_csv("done.csv")